#### General Code Setup
As a first step we:
- Import all the necessary libraries for our computation
- Setup some folders for the loading and saving of all necessary files and to run correctly EMUstack

#### Libraries imported
![xkcd python](https://imgs.xkcd.com/comics/python.png "I wrote 20 short programs in Python yesterday.  It was wonderful.  Perl, I'm leaving you.")
- [os](https://docs.python.org/3/library/os.html): used to setup some folder to run correctly the code
- [sys](https://docs.python.org/3/library/sys.html): used to make EMUstack accessible
- [numpy](https://numpy.org/): essentially matlab for python. We use it to manipulate matrix data
- [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html): to run the code in parallel
- [xarray](https://docs.xarray.dev/en/stable/): library to save labelled data along with the relevant metadata (important for reproducibility)
- [datetime](https://docs.python.org/3/library/datetime.html): we use it to create a timestamp for the saved files

In [ ]:
# libraries
import os
import numpy as np
import sys
import concurrent.futures
import xarray as xr
from datetime import datetime

# folder setup
home_dir = os.environ["HOME"]  # home folder
script_dir = os.getcwd()  # running file folder

#### Import EMUstack
In order to correctly load all the components of EMUstack we:
- make the EMUstack folder accessible
- load the EMUstack paths module to define the working folders
- load all the other EMUstack modules

In [ ]:
# setup EMUstack paths
sys.path.append(home_dir + "/programs/EMUstack/backend/")  # makes EMUstack accessible
import paths
paths.backend_path = home_dir + "/programs/EMUstack/backend/"  # location of EMUstack "engine"
paths.data_path = home_dir + "/programs/EMUstack/backend/data/"  # location of material files
paths.msh_path = script_dir + "/msh/"  # folder where we save the mesh files
paths.template_path = home_dir + "/programs/EMUstack/backend/fortran/msh/"  # folder containing the mesh templates

# import emustack
import objects  # EMUstack incident field and layer structures
import materials  # EMUstack available material library
import plotting  # EMUstack plotting routins
from stack import Stack  # EMUstack multilayer structures

#### Defining the inputs
We are going to calculate the transmission and reflectance spectrum of a Gold Nanodisk array. To do so we need to define the following groups of input parameters:
- **Geometrical parameters** of our multilayers, including thicknesses, disk size, etc...
- **Material parameters** such as the composition of each layer, substrate and superstrate included
- **Illumination parameters**, i.e. wavelengths, angles and polarizations of the incident field
- **EMUstack parameters**, i.e. parameters that are strictly connected with the inner workings of EMUstack

We will try to store all these input parameters inside [dictionaries](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), so that they can be handily exported later in order to improve the riproducibility of our calculations

In [ ]:
# light dictionary
light = {}
light['wl_min'] = 400.0  # minimum spectral wavelength
light['wl_max'] = 800.0  # maximum spectral wavelength
light['n_wl'] = 80  # spectral points
light['theta'] = 0.0
light['phi'] = 0.0
light['pol'] = 'TM'
light['max_order_PWs'] = 2  # number of plane waves

# materials dictionary
mats = {}
mats['Au'] = materials.Au
mats['Air'] = materials.Air

# Geometrical parameters
struct = {}
# nanodisk array parameters
struct['periodicity'] = "2D_array"  # we are modeling a 1d array (EMU)
struct['inc_shape'] = 'circle'  # nanodisks -> so we choose the circle shape (EMU)
struct['nd_radius'] = 100.0  # nanodisk radius
struct['nd_height'] = 100.0  # nanodisk height
struct['nd_period_x'] = 600.0  # nanodisk array x period
struct['nd_period_y'] = 600.0  # nanodisk array y period
struct['nd_inc_mat'] = 'Au'  # nanodisk material
struct['nd_back_mat'] = 'Air'  # background material in the nanodisk array
struct['nd_loss'] = True  # we must include losses in the computation (EMU)
# superstrate and substrate parameters
struct['sub_mat'] = 'Air'  # substrate material
struct['sup_mat'] = 'Air'  # superstrate material
struct['sub_loss'] = False  # no losses in the substrate (EMU)
struct['sup_loss'] = False  # no losses in the superstrate (EMU)

# Emustack parameters
emu = {}
emu['plotting_fields'] = False  # we do not want to plot the fields in this case 
emu['make_mesh_now'] = True  # we make the mesh, otherwise we must provide one with a filename
emu['force_mesh'] = True  # create mesh even if one already exist
emu['lc_bkg'] = 0.15  # background mesh finesse
emu['lc2'] = 2.0  # first mesh refinement
emu['lc3'] = 2.0  # second mesh refinement

# auxiliary vectors
v_wl = np.linspace(light['wl_min'], light['wl_max'], light['n_wl'])  # auxiliary wavelength vector
# objects.Light contains all the informations of the incident field, except for the polarization
light_list = [objects.Light(wl, max_order_PWs=light['max_order_PWs'], theta=light['theta'], phi=light['phi']) for wl in v_wl]

#### Initialize the single layers
In EMUstack each layer is initialized (and solved) separately and the solution is not influenced by its position inside the stack. There are two kind of layers:
- ThinFilm: layer that are not nanostructured and are simple to initialize
- NanoStruct: patterned layers which need more parameters to be initialized

In [ ]:
# initialize substrate and superstrate
substrate = objects.ThinFilm(
    period=struct["nd_period_x"], period_y=struct["nd_period_y"], height_nm="semi_inf", material=mats[struct["sub_mat"]], loss=struct["sub_loss"]
)
superstrate = objects.ThinFilm(
    period=struct["nd_period_x"], period_y=struct["nd_period_y"], height_nm="semi_inf", material=mats[struct["sup_mat"]], loss=struct["sup_loss"]
)

# initialize nanodisk array layer
NDs = objects.NanoStruct(
    struct['periodicity'],  # 1d or 2d periodicity
    struct["nd_period_x"],  # structure period, in only one period, then square lattice
    2.0 * struct["nd_radius"],  # diameter of the inclusion
    period_y=struct["nd_period_y"],  # period along y axis
    height_nm=struct["nd_height"],  # nanostructured layer thickness
    inclusion_a=mats[struct["nd_inc_mat"]],  # composition of the inclusion (Au)
    background=mats[struct["nd_back_mat"]],  # composition of the background (Air)
    loss=struct['nd_loss'],  # Loss flag, to decide if we include losses in the computation
    inc_shape=struct['inc_shape'],  # shape of the inclusion
    plotting_fields=emu['plotting_fields'],  # we preserve the eigenmodes to plot the fields?
    make_mesh_now=emu['make_mesh_now'],  # are we making the mesh now or not?
    force_mesh=emu['force_mesh'],  # are we making the mesh even if one already exists?
    lc_bkg=emu['lc_bkg'],  # mesh general finesse
    lc2=emu['lc2'],  # first level of mesh refinement
    lc3=emu['lc3'],  # second level of mesh refinement
)



#### Spectral computation
Now we build functions that:
- takes and incident field, i.e. and object.Light as input
- solves for the mode in each layer, where the FEM solver works in the nanostructured layer
- builds a stack with the solver layer, in the order bottom -> top
- returns the stack
- take the returned stack as input and compute R,T,A spectra as output

Then we run a calculation solving the problem at each wavelength

In [ ]:
# EMUstack Function
def simulate_stack(light):

    # evaluate each layer individually
    sim_NDs = NDs.calc_modes(light)  # solving the eigenproblem for the nanodisk layer
    sim_superstrate = superstrate.calc_modes(light)  # solving the plane wave problem for the superstrate
    sim_substrate = substrate.calc_modes(light)  # solving the plane wave problem for the substrate

    # build the stack with the solved layers and return it
    stackSub = Stack((sim_substrate, sim_NDs, sim_superstrate))

    return stackSub


def rta_spectra(stacks_list, pol="TE"):

    # create empty lists to store the partial results
    a_list = []
    t_list = []
    r_list = []
    # iterate over all the stacks to extract the results
    for stack in stacks_list:
        stack.calc_scat(pol = pol)
        a_list.extend(stack.a_list)
        t_list.extend(stack.t_list)
        r_list.extend(stack.r_list)

    # extract the total results
    layers_steps = len(stacks_list[0].layers) - 1
    a_tot      = []
    t_tot      = []
    r_tot      = []
    for i in range(len(stacks_list)):
        a_tot.append(float(a_list[layers_steps-1+(i*layers_steps)]))
        t_tot.append(float(t_list[layers_steps-1+(i*layers_steps)]))
        r_tot.append(float(r_list[i]))

    return np.array(r_tot),np.array(t_tot),np.array(a_tot)

In [ ]:
# executor distributes the computation on all the available cores
with concurrent.futures.ProcessPoolExecutor() as executor: 
    stacks_list = list(executor.map(simulate_stack, light_list))

In [ ]:
# spectra computation
v_R, v_T, v_A = rta_spectra(stacks_list, pol=light['pol'])


#### Saving data and metadata
We recast all the data in a format suitable for saving. We also save all the computation parameters as metadata, in order to make the computation of the spectrum reproducible

In [ ]:
# build output filename
now = datetime.now()  # what time is it?
timestamp = now.strftime("%d%m%y_%H%M%S")  # build timestamp string as day,month,year _ hour,minute,second
out_filename = (
    "EMU_AuNdArraySpectra_"
    + struct["inc_shape"]
    + "_px"
    + str(struct["nd_period_x"])
    + "_py"
    + str(struct["nd_period_y"])
    + "_r"
    + str(struct["nd_radius"])
    + "_h"
    + str(struct["nd_height"])
    + "_"
    + timestamp
)

# save data to xarray for future use
emu_spectra = xr.DataArray(np.column_stack((v_R,v_T,v_A)), coords=[v_wl, ['R','T','A']], dims=["wavelength", "spectrum"])
emu_spectra.attrs['light'] = str(light)
emu_spectra.attrs['struct'] = str(struct)
emu_spectra.attrs['emu'] = str(emu)
emu_spectra.attrs['mats'] = str(mats)
emu_spectra.to_netcdf('data/' + out_filename +  '.nc')